# Agente Bayesiano para Nine Mens Morri's

## Creación del vocabulario
Primero vamos a seccionar en 4 la cadena de 24 caracteres, con el fin de que al momento de hacer el conteo junto con la base de datos podamos realizar el conteo.

In [4]:
# Creación del vocabulario
from itertools import product

estadoCasilla = ["O", "M", "E"]


palabras = map(''.join, product(estadoCasilla, repeat=6))

# Este tal cual son las palabras pero para poder hacer el conteo de forma más sencilla
# decidi dejar el vocabulario como un diccionario que apunta a una lista. Esta lista
# representa las 24 posiciones de las casillas.
vocabulario = {palabra: [float(0)] * 24 for palabra in palabras}

<a target="_blank" href="https://colab.research.google.com/github/Albert236/Proyecto-Final-IA/blob/main/ProyectoFinal.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
  

## Conteo
Una vez realizado el conteo debemos de realizar el conteo con la base de datos, es por eso que se puede clonar el repositorio de github desde aquí.

In [1]:
!git clone https://github.com/Albert236/Proyecto-Final-IA.git

Cloning into 'Proyecto-Final-IA'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (17/17), 221.30 KiB | 2.33 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
# Nos movemos a la carpeta
import os
os.chdir("Proyecto-Final-IA")

In [6]:
# Esto nos va a ayudar a reliazar el conteo
CANTIDAD = 15000
indiceMovimientos = {"a7":0,"a4":1,"a1":2,"b6":3,"b4":4,"b2":5,"c5":6,"c4":7,"c3":8,"d7":9,"d6":10,"d5":11,"d3":12,"d2":13, "d1":14, "e5":15,"e4":16,"e3":17,"f6":18,"f4":19,"f2":20,"g7":21,"g4":22,"g1":23}

In [10]:
import copy


# ti representa la palabra dentro del vocabulario
t1 = copy.deepcopy(vocabulario)
t2 = copy.deepcopy(vocabulario)
t3 = copy.deepcopy(vocabulario)
t4 = copy.deepcopy(vocabulario)

with open("fase1DB.txt") as f:
    lines = f.readlines()
    contador = 0
    for line in lines:
      tablero = line.split("-")[0]
      movimiento = line.split("-")[1]

      for palabra in vocabulario.keys():
          if palabra ==  tablero[0:6]:
              t1[palabra][indiceMovimientos[movimiento[:2]]] += 1.0
          elif palabra ==  tablero[6:12]:
              t2[palabra][indiceMovimientos[movimiento[:2]]] += 1.0
          elif palabra ==  tablero[12:18]:
              t3[palabra][indiceMovimientos[movimiento[:2]]] += 1.0
          elif palabra ==  tablero[18:24]:
              t4[palabra][indiceMovimientos[movimiento[:2]]] += 1.0

      contador += 1

      if contador == CANTIDAD:
        break

# Se realiza la siguiente descomposición parametrica
for key in t1.keys():
  for i in range(24):
    t1[key][i] = (1 + t1[key][i]) / (CANTIDAD + 729)
for key in t2.keys():
  for i in range(24):
    t2[key][i] = (1 + t2[key][i]) / (CANTIDAD + 729)
for key in t3.keys():
  for i in range(24):
    t3[key][i] = (1 + t3[key][i]) / (CANTIDAD + 729)
for key in t4.keys():
  for i in range(24):
    t4[key][i] = (1 + t4[key][i]) / (CANTIDAD + 729)

## Jugada por estado del tablero
Ahora podemos que dado un estado del juego, cuál sería la mejor jugada?

In [11]:
"""
Función que dado un tablero infiera la mejor jugada, sin importar si esta es legal o no
"""
def mejorJugada(tablero):
  ti = [tablero[0:6], tablero[6:12], tablero[12:18], tablero[18:24]]
  distribJi = {"a7":1.0,"a4":1.0,"a1":1.0,"b6":1.0,"b4":1.0,"b2":1.0,"c5":1.0,"c4":1.0,"c3":1.0,"d7":1.0,"d6":1.0,"d5":1.0,"d3":1.0,"d2":1.0, "d1":1.0, "e5":1.0,"e4":1.0,"e3":1.0,"f6":1.0,"f4":1.0,"f2":1.0,"g7":1.0,"g4":1.0,"g1":1.0}

  for key in distribJi.keys():
    distribJi[key] = t1[ti[0]][indiceMovimientos[key]]*t2[ti[1]][indiceMovimientos[key]]*t3[ti[2]][indiceMovimientos[key]]*t4[ti[3]][indiceMovimientos[key]]

  return max(distribJi, key=distribJi.get)

In [12]:
mejorJugada("OEMOOOOOOOOOOOOOOOOEOMOE7623")

'f4'

In [14]:
# vamos a comprobar con los datos de la prueba la cantidad de aciertos que genera
with open("datosPrueba.txt", "r") as file:
  lines = file.readlines()
  contador = 0

  for line in lines:
    tablero = line.split("-")[0]
    jugada = line.split("-")[1][:2]

    if jugada == mejorJugada(tablero):
      contador += 1

  print(contador/len(lines))

0.21006178287731686


Como se puede apreciar en la que se acierta es bajísima, pero recordemos que esta función sigue incompleta, vamos a ver que pasa cuando se buscan únicamente jugadas legales.

In [45]:
def mejorJugadaLegal(tablero):
  ti = [tablero[0:6], tablero[6:12], tablero[12:18], tablero[18:24]]
  distribJi = {"a7":1.0,"a4":1.0,"a1":1.0,"b6":1.0,"b4":1.0,"b2":1.0,"c5":1.0,"c4":1.0,"c3":1.0,"d7":1.0,"d6":1.0,"d5":1.0,"d3":1.0,"d2":1.0, "d1":1.0, "e5":1.0,"e4":1.0,"e3":1.0,"f6":1.0,"f4":1.0,"f2":1.0,"g7":1.0,"g4":1.0,"g1":1.0}

  for key in distribJi.keys():
    distribJi[key] = t1[ti[0]][indiceMovimientos[key]]*t2[ti[1]][indiceMovimientos[key]]*t3[ti[2]][indiceMovimientos[key]]*t4[ti[3]][indiceMovimientos[key]]

  posiblesJugadas = sorted(distribJi, key=distribJi.get, reverse=True)

  for intento in posiblesJugadas:
    if esLegal(tablero, intento):
      return intento

  return None

def esLegal(tablero, intento):
  if tablero[indiceMovimientos[intento]] != "O":
    return False
  else:
    return True


In [49]:
# Veamos si la cantidad de movimientos acertados aumenta
with open("datosPrueba.txt", "r") as file:
  lines = file.readlines()
  contador = 0

  for line in lines:
    tablero = line.split("-")[0]
    jugada = line.split("-")[1][:2]

    if jugada == mejorJugadaLegal(tablero):
      contador += 1

  print(contador/len(lines))
# al parecer no lo hace

3399
0.17917034421888792
